In [1]:
import pandas as pd
from difflib import SequenceMatcher


In [ ]:
# === 1. 加载定性与定量数据 ===
df_q = pd.read_csv("xxx_定量_匹配结果.csv")  # 替换为你的定量表路径
df_ql = pd.read_csv("xxx_定性_匹配结果.csv")  # 替换为你的定性表路径

# === 2. 统一关键词小写处理 ===
df_q["keyword_lower"] = df_q["keyword"].str.lower().str.strip()
df_ql["keyword_lower"] = df_ql["keyword"].str.lower().str.strip()

# === 3. 清洗与合并 ===
df_q["value_clean"] = pd.to_numeric(df_q["value"].replace('%', '', regex=True), errors="coerce")
df_q_clean = df_q.dropna(subset=["value_clean"])[["keyword_lower", "value_clean"]]
df_q_clean["类型"] = "定量"

df_ql["value_clean"] = pd.to_numeric(df_ql["total_frequency"], errors="coerce")
df_ql_clean = df_ql.dropna(subset=["value_clean"])[["keyword_lower", "value_clean"]]
df_ql_clean["类型"] = "定性"

# 定量优先合并
quant_keywords = set(df_q_clean["keyword_lower"])
df_ql_clean = df_ql_clean[~df_ql_clean["keyword_lower"].isin(quant_keywords)]
df_combined = pd.concat([df_q_clean, df_ql_clean], ignore_index=True)

# === 4. 计算每个关键词均值 ===
df_avg = df_combined.groupby("keyword_lower").agg({"value_clean": "mean"}).reset_index()
df_avg.columns = ["关键词", "值"]

# === 5. 加载 ESG 指标 ===
esg = pd.ExcelFile("ESG评价体系0322.xlsx")  # 替换为你的 ESG 参考表格
e_list = esg.parse("Environment")["Metric"].dropna().tolist()
s_list = esg.parse("Social")["Metric"].dropna().tolist()
g_list = esg.parse("Governance")["Metric"].dropna().tolist()

# === 6. 匹配函数 ===
def fuzzy_match_metric(keywords, targets, threshold=0.4):
    matched = []
    for kw in keywords:
        best_score, best_match = 0, None
        for tgt in targets:
            score = SequenceMatcher(None, kw.lower(), tgt.lower()).ratio()
            if score > best_score:
                best_score, best_match = score, tgt
        if best_score >= threshold:
            matched.append((kw, best_match, best_score))
    return matched

def build_match_table(matched_list, df_val, category):
    val_map = dict(zip(df_val["关键词"].str.lower(), df_val["值"]))
    return pd.DataFrame([{
        "ESG类别": category,
        "匹配关键词": kw,
        "ESG指标": mt,
        "匹配关键词值": val_map.get(kw.lower(), None)
    } for kw, mt, _ in matched_list])

# === 7. 执行匹配 ===
matched_e = fuzzy_match_metric(df_avg["关键词"], e_list)
matched_s = fuzzy_match_metric(df_avg["关键词"], s_list)
matched_g = fuzzy_match_metric(df_avg["关键词"], g_list)

df_e = build_match_table(matched_e, df_avg, "E")
df_s = build_match_table(matched_s, df_avg, "S")
df_g = build_match_table(matched_g, df_avg, "G")

# === 8. 标准化格式（添加强度与理由） ===
def standardize(df):
    df = df.rename(columns={"ESG指标": "指标"})
    df["匹配强度"] = "语义匹配"
    df["匹配理由"] = "基于关键词含义的模糊匹配"
    return df[["指标", "匹配关键词", "匹配强度", "匹配理由", "匹配关键词值"]]

df_e_std = standardize(df_e)
df_s_std = standardize(df_s)
df_g_std = standardize(df_g)

# === 9. 相同指标均值去重 ===
def dedup_by_metric(df):
    return df.groupby("指标").agg({
        "匹配关键词值": "mean",
        "匹配关键词": "first",
        "匹配强度": "first",
        "匹配理由": "first"
    }).reset_index()[["指标", "匹配关键词", "匹配强度", "匹配理由", "匹配关键词值"]]

df_e_final = dedup_by_metric(df_e_std)
df_s_final = dedup_by_metric(df_s_std)
df_g_final = dedup_by_metric(df_g_std)

# === 10. 导出结果 ===
with pd.ExcelWriter("公司名_ESG_匹配带值_标准格式去重均值版.xlsx") as writer:
    df_e_final.to_excel(writer, sheet_name="E环境类", index=False)
    df_s_final.to_excel(writer, sheet_name="S社会类", index=False)
    df_g_final.to_excel(writer, sheet_name="G治理类", index=False)


In [ ]:
import pandas as pd

# === 1. 指定每个公司的 ESG 文件路径 ===
files = {
    "Alcon": "Alcon_ESG_匹配带值_标准格式去重均值版.xlsx",
    "BMS": "BMS_ESG_匹配带值_逗号空格分隔_严格匹配版.xlsx",
    "EliLilly": "EliLilly_ESG_匹配带值_标准格式去重均值版.xlsx",
    "HengRui": "HengRui_ESG_匹配带值_标准格式去重均值版.xlsx"
    # 可继续添加公司
}

# === 2. 提取每家公司 E sheet 中的指标值 ===
company_rows = []

for company, path in files.items():
    df = pd.read_excel(path, sheet_name="E环境类", usecols=["指标", "匹配关键词值"])
    df_clean = df.dropna(subset=["指标", "匹配关键词值"])
    company_row = df_clean.set_index("指标")["匹配关键词值"].to_dict()
    company_row["公司"] = company
    company_rows.append(company_row)

# === 3. 构造 DataFrame，"公司" 为行，指标为列 ===
df_result = pd.DataFrame(company_rows)
df_result = df_result.set_index("公司")

# === 4. 导出结果 ===
df_result.to_excel("ESG_环境类_公司为行_指标为列_聚合总表.xlsx")
